## Import

In [1]:
from PublicDataReader import Kbland

import xlwings as xw
import pandas as pd
import numpy as np
import time
import os

# path setting
if not os.path.isdir('../../datasets/KB/'):
    os.mkdir('../../datasets/KB/')
    
os.chdir('../../datasets/KB/')

* functions

In [2]:
# 파일 불러오기
def read_data(file):
    try:
        output = pd.read_excel(file)
    except:
        def read_xlsx(name):
            instance = xw.App(visible=False)
            xlsx_data = xw.Book(name).sheets[0]
            df = xlsx_data.range('A1').options(pd.DataFrame, index = False, expand = 'table').value
            instance.quit()
            instance.kill()
            return df
        output = read_xlsx(file)
    return output

import PublicDataReader as pdr
code = pdr.code_bdong()
code.head()

출처: 행정기관(행정동) 및 관할구역(법정동) 변경내역(2023.10.30. 시행)


,시도코드,시도명,시군구코드,시군구명,법정동코드,읍면동명,동리명,생성일자,말소일자
0,11,서울특별시,11000,,1100000000,,,19880423,
1,11,서울특별시,11110,종로구,1111000000,,,19880423,
2,11,서울특별시,11110,종로구,1111010100,청운동,,19880423,
3,11,서울특별시,11110,종로구,1111010200,신교동,,19880423,
4,11,서울특별시,11110,종로구,1111010300,궁정동,,19880423,


* 매매 가격 지수

In [3]:
def kbland_sales_price_idx1(a,b,c):
    # 매매가격지수
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EA%B0%80%EA%B2%A9%EC%A7%80%EC%88%981
    # 주간/월간 아파트.단독.연립.주택종합 매매/전세 가격지수
    api = Kbland()
    params = {
        "월간주간구분코드": a,
        "매물종별구분": b,
        "매매전세코드": c,
    }
    df = kb_api.get_price_index(**params)
    return df

def kbland_sales_price_change1(a,b,c):
    # 매매가격지수 증감률
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EA%B0%80%EA%B2%A9%EC%A7%80%EC%88%98%EC%A6%9D%EA%B0%90%EB%A5%A0
    params = {
    "월간주간구분코드": a,
    "매물종별구분": b,
    "매매전세코드": c,
    }
    df = kb_api.get_price_index_change_rate(**params)
    return df

def get_mnthly_price_idx1():
    # 월간 아파트 월세가격지수
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EA%B0%80%EA%B2%A9%EC%A7%80%EC%88%982
    return kb_api.get_monthly_apartment_wolse_index()

def get_sales_price_idx1():
    # 매매가격지수
    # 매매가격지수 증감률
    loop1 = ['01', '02'] # 월간, 주간
    loop2 = ['01', '08', '09', '98'] # 아파트, 연립, 단독, 주택종합
    loop3 = ['01', '02'] # 매매, 전세
    
    start = time.time()

    sales_dfs1 = []
    sales_dfs2 = []
    for a in loop1:
        for b in loop2:
            for c in loop3:
                sales_dfs1.append(kbland_sales_price_idx1(a,b,c))
                sales_dfs2.append(kbland_sales_price_change1(a,b,c)) # 증감률

    sales_df1 = pd.concat(sales_dfs1)
    sales_df2 = pd.concat(sales_dfs2)
    sales_df = pd.merge(sales_df1, sales_df2, on = ['월간주간구분', '매물종별구분', '거래구분', '지역코드', '지역명', '날짜'])
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return sales_df

def get_top50_idx():
    # KB선도아파트 50 지수
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EA%B0%80%EA%B2%A9%EC%A7%80%EC%88%983
    df = kb_api.get_lead_apartment_50_index()
    return df

def get_ratios():
    # 전세가격비율
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EC%A0%84%EC%84%B8%EA%B0%80%EA%B2%A9%EB%B9%84%EC%9C%A81
    start = time.time()
    
    loop2 = ['01', '08', '09', '98'] # 아파트, 연립, 단독, 주택종합

    ratios = []
    for a in loop2:
        params = {
            "매물종별구분": a,
        }
        ratios.append(kb_api.get_jeonse_price_ratio(**params))

    ratio_df = pd.concat(ratios)
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return ratio_df

def get_ratios2():
    # 전월세전환율
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EC%A0%84%EC%84%B8%EA%B0%80%EA%B2%A9%EB%B9%84%EC%9C%A82
    return kb_api.get_jeonwolse_conversion_rate()

def get_market_index():
    # 시장동향/설문조사
    # https://github.com/WooilJeong/PublicDataReader/blob/main/assets/docs/kbland/Kbland.md#%EC%8B%9C%EC%9E%A5%EB%8F%99%ED%96%A5%EC%84%A4%EB%AC%B8%EC%A1%B0%EC%82%AC
    start = time.time()
    
    loop1 = ['01', '02', '03', '04', '05', '06'] # 매수우위지수, 매매거래활발지수, 전세수급지수, 전세거래활발지수, 매매가격전망지수, 전세가격전망지수
    loop2 = ['01', '02'] # 월간, 주간

    start = time.time()

    market_idx = []
    for a in loop1:
        for b in loop2:
            params = {
                "메뉴코드": a,
                "월간주간구분코드": b,
            }
            df = kb_api.get_market_trend(**params)
            temp_df = df.loc[:, :'날짜']
            temp_df['지수'] = df.iloc[:, -1]
            
            market_idx.append(temp_df)

    market_df = pd.concat(market_idx)
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return market_df

def get_per_price_df():
    # 면적별 가격 지수
    loop1 = ['01', '02'] # 월간, 주간
    loop2 = ['01', '08', '09', '98'] # 아파트, 연립, 단독, 주택종합
    loop3 = ['01', '02'] # 전용면적별(구), 전용면적별)
    loop4 = ['01', '02'] # 매매, 전세
    
    start = time.time()

    per_price = []
    for a in loop1:
        for b in loop2:
            for d in loop4:
                params = {
                    "월간주간구분코드": a,
                    "매물종별구분": b,
                    "면적별코드": '02',
                    "매매전세코드": d,
                }
                df = kb_api.get_price_index_by_area(**params)
                per_price.append(df)

    pp_df = pd.concat(per_price)
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return pp_df

def get_price_df():
    # 지역별 평균 가격
    # 지역별 제곱미터당 평균 가격
    # 지역별 중위 가격
    loop1 = ['01', '02'] # 월간, 주간
    loop2 = ['01', '08', '09', '98'] # 아파트, 연립, 단독, 주택종합
    
    start = time.time()

    avg_price = []
    avg_land_price = []
    median_price = []
    for a in loop1:
        for b in loop2:
            params = {
                "매물종별구분": b,
                "매매전세코드": a,
            }
            avg_price.append(kb_api.get_average_price(**params))
            avg_land_price.append(kb_api.get_average_price_per_squaremeter(**params))
            median_price.append(kb_api.get_median_price(**params))

    avg_df = pd.concat(avg_price)
    avg_land_df = pd.concat(avg_land_price)
    median_df = pd.concat(median_price)
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return avg_df, avg_land_df, median_df

def get_pir_df():
    # 소득 대비 주택 가격
    loop1 = ['01', '02'] # PIR, J-PIR
    
    start = time.time()

    pir_list = []
    for a in loop1:
        params = {
            "메뉴코드": a,
        }
        pir_list.append(kb_api.get_pir(**params))

    pir_df = pd.concat(pir_list)
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return pir_df

In [4]:
kb_api = Kbland()
sales_df = get_sales_price_idx1() # 매매,전세 가격 지수 및 증감률 (전국-도)
mnthly_df = get_mnthly_price_idx1() # 월간 아파트 월세가격 지수 (수도권, 도)
top50 = get_top50_idx() # 선도 50 지수 (전국)
ratio_df = get_ratios() # 매매 가격 대비 전세 가격 (전국-도)
ratio_df2 = get_ratios2() # 전월세 전환률 (수도권, 도)
market_df = get_market_index() # 매수우위, 전세수급, 거래활발, 전망 지수 (전국-도)
pp_df = get_per_price_df() # 전용 면적별 가격 지수 (전국-도)
avg_df, avg_land_df, median_df = get_price_df() # 지역별 평균 및 중위 가격 (전국-도)
pir_df = get_pir_df() # 소득 대비 주택 가격 (전국, 서울특별시)
hai_df = kb_api.get_hai() # 주택 구입 부담 지수 (전국 전체, 수도권, 기타지방)
kb_poten_df = kb_api.get_kb_housing_purchase_potential_index() # KB구매 잠재력 지수 (서울, 경기, 인천)
kb_loan_df = kb_api.get_apartment_mortgage_loan_pir() # KB구매 잠재력 지수 (서울, 경기, 인천)

수행시간: 72.249120 초
수행시간: 11.687028 초
수행시간: 60.000242 초
수행시간: 44.112596 초
수행시간: 41.538734 초
수행시간: 3.556897 초


In [ ]:
params = {
    "월간주간구분코드": "01",
    "매물종별구분": "01",
    "매매전세코드": "01",
    "지역코드": "11",
    "기간": "1",
}
df = kb_api.get_price_index_change_rate(**params)
df.tail()

,월간주간구분,매물종별구분,거래구분,지역코드,지역명,날짜,가격지수증감률
333,월간,아파트,매매,1174000000,강동구,2023-06-01,-0.094459
334,월간,아파트,매매,1174000000,강동구,2023-07-01,0.418118
335,월간,아파트,매매,1174000000,강동구,2023-08-01,0.445742
336,월간,아파트,매매,1174000000,강동구,2023-09-01,0.450658
337,월간,아파트,매매,1174000000,강동구,2023-10-01,0.356318


* 파일 저장